In [1]:
import random
import time
import redis
import json
# 與MySQL資料庫連線取得其中資料
# !pip install mysql-connector-python
import mysql.connector
r = redis.Redis(host='192.168.80.166', port=6379)

In [2]:
# function from other team member
# 使用函式取得不同運動類型、強度及城市的運動路線
def road(type3,level,city):
    db = mysql.connector.connect(user='root', password='1234', host='10.120.28.5',port='3306', database="final",charset = 'utf8')
    cursor = db.cursor()    
    
    type3_dict = {"climb":" > 150 ","run":" < 150 "}
    x = type3_dict.get(type3)
    
    level_dict ={"strong":" '高' ","medium":" '中' ","weak":" '低' "}
    y = level_dict.get(level)
    
    city_dict = {"TPE":"%台北%", "NTPC":"%新北%", "KEL":"%基隆%", "TYN":"%桃園%", "HSZ":"%新竹%",
                "ZMI":"%苗栗%", "ILA":"%宜蘭%", "TXG":"%台中%", "CHW":"%彰化%", "NTC":"%南投%", 
                 "YUN":"%雲林%", "CYI":"%嘉義%", "TNN":"%台南%", "KHH":"%高雄%", "PIF":"%屏東%",
                "PEH":"%澎湖%"}
    z = city_dict.get(city)
    
    cursor.execute("SELECT final.mountain_basic.url, final.mountain_basic.標題, final.mountain_basic.評分, final.mountain_basic.里程, final.mountain_basic.海拔高度, final.mountain_basic.難易度, final.mountain_city.所在縣市 FROM final.mountain_basic LEFT JOIN final.mountain_city ON final.mountain_basic.標題 = final.mountain_city.標題 WHERE final.mountain_basic.高度落差"+ x +"AND final.mountain_city.所在縣市 LIKE '"+ z +"' AND final.mountain_basic.難易度 LIKE" + y)  
    
    res_list = []
    for x in cursor:
        # url
        a = x[0]
        # 標題
        b = x[1]
        # 評分
        c = x[2]
        # 里程
        d = x[3]
        # 海拔高度
        e = x[4]
        # 難易
        f = x[5]
        # 縣市
        g = x[6]
        res = {'url': a, 'name': b, 'score':c, 'mileage':d,'high':e, 'level':f, 'city':g}
        res_list.append(res)
    db.close()
    return res_list

In [ ]:
# saving random one route of different type, strength ,city
# 在redis中隨機儲存不同運動類型、強度、城市的資料
city_dict = {"NTPC":"新北", "ZMI":"苗栗", "NTC":"南投"}
while True:
    for city in ["NTPC", "ZMI", "NTC"]:
        for strength in ["strong", "medium", "weak"]:   
            r_info = road("run", strength, city)
            # if result of route doesn't exist
            # 如果無此路線資料
            if r_info == []:
                r.hmset("run"+"_"+strength+"_"+city, {city: city_dict[city], strength: strength, 'info': 'Not find the route'})
            # result for route is a list, random.choice one of list to set in redis
            # 使用random.choice在相同運動類型、強度、城市的list擇一
            elif r_info != []:
                r.hmset("run"+"_"+strength+"_"+city, {city: city_dict[city], strength: strength, 'info': str(random.choice(r_info))})

            m_info = road("climb", strength, city)
            if m_info == []:
                r.hmset("climb"+"_"+strength+"_"+city, {city: city_dict[city], strength: strength, 'info': 'Not find the route'})
            elif m_info != []:
                r.hmset("climb"+"_"+strength+"_"+city, {city: city_dict[city], strength: strength, 'info': str(random.choice(m_info))})
            print(city, '|', city_dict[city], '|', strength )
            # 測試取得redis中資料，儲存在info的資料為字串格式的字典
            print("running:", r.hmget("run"+"_"+strength+"_"+city, [city, strength, 'info'])[2].decode())
            print("mountaineering:", r.hmget("climb"+"_"+strength+"_"+city, [city, strength, 'info'])[2].decode())
            print('--------------------------------------------------STRENGTH--------------------------------------------------')
            time.sleep(600)
        print('--------------------------------------------------CITY--------------------------------------------------')
        time.sleep(600)
    print('--------------------------------------------------UPDATE--------------------------------------------------')
    time.sleep(600)

NTPC | 新北 | strong
running: {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1603', 'name': '板橋新月橋天空步道、新海人工濕地', 'score': 4, 'mileage': '10.4公里', 'high': '1~14公尺', 'level': '高', 'city': '新北市新莊區'}
mountaineering: {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1218', 'name': '陽明山東西大縱走(超健腳加強版)', 'score': 5, 'mileage': '39.2公里', 'high': '5~1120公尺', 'level': '高', 'city': '新北市淡水區'}
--------------------------------------------------STRENGTH--------------------------------------------------
NTPC | 新北 | medium
running: {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=446', 'name': '拉拉山神木群步道', 'score': 4, 'mileage': '4公里', 'high': '1550~1650公尺', 'level': '中', 'city': '新北市烏來區'}
mountaineering: {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1015', 'name': '荖蘭山、隆隆山、萊萊山登山步道', 'score': 4, 'mileage': '10.5公里', 'high': '15~433公尺', 'level': '中', 'city': '新北市貢寮區'}
--------------------------------------------------STRENGTH---------------------